In [7]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from matplotlib.gridspec import GridSpec
import lightgbm as lgb
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
from sklearn.kernel_ridge import KernelRidge
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.feature_selection import SelectFromModel


# Ignore ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [8]:
imputed_data = pd.read_csv("imputed_data_handle_multicollinearity.csv")
imputed_data=imputed_data.drop('Unnamed: 0',axis=1)
imputed_data.head()

,GrLivArea,SalePrice,OverallQual,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,BsmtFinType2_Unknown,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y
0,1049.0,139500.0,5.0,1984.0,1984.0,552.0,393.0,104.0,1049.0,1049.0,...,0,0,0,1,0,0,0,0,1,1
1,1049.0,139500.0,5.0,1984.0,1984.0,552.0,393.0,104.0,1049.0,1049.0,...,0,0,0,1,0,0,0,0,0,1
2,1049.0,139500.0,5.0,1984.0,1984.0,552.0,393.0,104.0,1049.0,1049.0,...,0,0,0,1,0,0,0,0,1,1
3,1001.0,124900.0,5.0,1930.0,2007.0,737.0,0.0,100.0,837.0,1001.0,...,0,0,0,0,0,0,0,0,1,1
4,1001.0,124900.0,5.0,1930.0,2007.0,737.0,0.0,100.0,837.0,1001.0,...,0,0,0,1,0,0,0,0,0,1


Running KFold: 100%|█████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  1.83it/s]


In [10]:
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR

# Separate the independent variables (features) from the dependent variable (target)
X = imputed_data.drop('SalePrice', axis=1)
y = imputed_data['SalePrice']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the model for feature selection
selector_model = SVR(kernel='linear')

# Fit the model for feature selection
selector = SelectFromModel(selector_model)
selector.fit(X_train, y_train)

# Get the selected features
selected_features = X_train.columns[selector.get_support()].tolist()

# Select the selected features from the train and test sets
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Define the model for regression
model = SVR(kernel='linear')

# Set up KFold with shuffle=True
kf = KFold(n_splits=5, shuffle=True)

# Initialize empty lists to store R2 scores
train_r2_scores = []
test_r2_scores = []

# Loop through the folds with a tqdm progress bar
for train_index, test_index in tqdm(kf.split(X_train_selected), total=kf.n_splits, desc='Running KFold'):
    X_train_fold, X_test_fold = X_train_selected.iloc[train_index], X_train_selected.iloc[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    # Fit the model on the selected features and calculate R2 on the train and test sets
    model.fit(X_train_fold, y_train_fold)
    train_r2 = model.score(X_train_fold, y_train_fold)
    test_r2 = model.score(X_test_fold, y_test_fold)
    
    # Append the R2 scores to the lists
    train_r2_scores.append(train_r2)
    test_r2_scores.append(test_r2)

# Plot the results
num_features_list = list(range(1, len(selected_features) + 1))
test_r2_list = []
for i in range(len(num_features_list)):
    selected_features_i = selected_features[:i+1]
    X_train_selected_i = X_train[selected_features_i]
    X_test_selected_i = X_test[selected_features_i]
    model.fit(X_train_selected_i, y_train)
    test_r2_i = model.score(X_test_selected_i, y_test)
    test_r2_list.append(test_r2_i)

plt.plot(num_features_list, test_r2_list)
plt.xlabel('Number of Features')
plt.ylabel('R2 Score')
plt.title('Number of Features vs R2 Score')
plt.xticks(range(1, len(selected_features)+1))
plt.ylim(0.5, 1)
plt.xlim(1,len(selected_features))
plt.show()


In [11]:
# Create a dataframe to store the results
results_df = pd.DataFrame({'importance_rank': range(1, len(selected_features)+1),
                           'feature_name': selected_features,
                           'test_r2': test_r2_list})

# Sort the dataframe by test_r2 in ascending order
results_df = results_df.sort_values(by='test_r2')
results_df['importance_rank'] = range(1, len(selected_features)+1)

# Print the ranked feature list
print(results_df[['importance_rank', 'feature_name', 'test_r2']].head(10))


    importance_rank  feature_name   test_r2
0                 1     GrLivArea  0.459400
1                 2   OverallQual  0.473128
2                 3     YearBuilt  0.611849
3                 4  YearRemodAdd  0.626410
4                 5    BsmtFinSF1  0.709263
5                 6   TotalBsmtSF  0.749399
10                7  KitchenAbvGr  0.751135
9                 8      HalfBath  0.751143
8                 9  BsmtHalfBath  0.751181
7                10  BsmtFullBath  0.751187


In [12]:
# Save the results in the current working directory
results_df.to_csv('results_svr.csv', index=False)